In [2]:
import tensorflow as tf
import numpy as np

## 加载MNIST数据集

分别加载MNIST训练集、测试集、验证集

In [5]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train = mnist.train.images, mnist.train.labels
X_test, y_test = mnist.test.images, mnist.test.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
print("Image Shape: {}".format(X_train.shape))
print("label Shape: {}".format(y_train.shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

Image Shape: (55000, 784)
label Shape: (55000, 10)

Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


## VGG16网络结构

In [8]:
def get_conv_filter(name):
    return tf.constant(data_dict[name][0], name='filter')

In [9]:
def get_biases(name):
    return tf.constant(data_dict[name][1], name='biases')

In [10]:
def get_fc_weight(name):
    return tf.constant(data_dict[name][0], name='weights')

In [6]:
def conv_layer(x_input, name):
    with fg.variable_scope(name):
        filt = get_conv_filter(name)
        
        layer = tf.nn.conv2d(x_input, filt, [1, 1, 1, 1], padding='SAME')
        
        conv_biases = get_biases(name)
        layer = tf.nn.bias_add(layer, conv_biases)
        
        layer = tf.nn.relu(layer)
        return layer

In [7]:
def fc_layer(x_input, name):
    with tf.variable_scope(name):
        shape = x_input.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        x = tf.reshape(x_input, [-1, dim])
        
        w = get_fc_weight(name)
        b = get_biases(name)
        
        fc = tf.nn.bias_add(tf.matmul(x, w), b)
    return fc

In [ ]:
def max_pool_layer(x_input, name):
    return tf.nn.max_pool(x_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [ ]:
def avg_pool_layer(x_input, name):
    return tf.nn.avg_pool(x_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [4]:
def VGG16(x):
    
    # 224x224x64
    conv1_1 = conv_layer(bgr, 'conv1_1')
    conv1_2 = conv_layer(conv1_1, 'conv1_2')
    pool1 = max_pool_layer(conv1_2, 'pool1')
    
    # 
    conv2_1 = conv_layer(pool1, 'conv2_1')
    conv2_2 = conv_layer(conv2_1, 'conv2_2')
    pool2 = max_pool_layer(conv2_2, 'pool2')
    
    conv3_1 = conv_layer(pool2, 'conv3_1')
    conv3_2 = conv_layer(conv3_1, 'conv3_2')
    conv3_3 = conv_layer(conv3_2, 'conv3_3')
    pool3 = max_pool_layer(conv3_3, 'pool3')
    
    conv4_1 = conv_layer(pool3, 'conv4_1')
    conv4_2 = conv_layer(conv4_1, 'conv4_2')
    conv4_3 = conv_layer(conv4_2, 'conv4_3')
    pool4 = max_pool_layer(conv4_3, 'pool4')
    
    conv5_1 = conv_layer(pool4, 'conv5_1')
    conv5_2 = conv_layer(conv5_1, 'conv5_2')
    conv5_3 = conv_layer(conv5_2, 'conv5_3')
    pool5 = max_pool_layer(conv5_3, 'pool5')
    
    fc6 = fc_layer(pool, 'fc6')
    assert fc6.get_shape().as_list()[1:] == [4096]
    relu6 = tf.nn.relu(fc6)
    
    fc7 = fc_layer(relu7, 'fc7')
    relu7 = tf.nn.relu(fc7)
    
    fc8 = fc_layer(relu7, 'fc8')
    
    prob = tf.nn.softmax(fc8, name='prob')